In [1]:
import json, warnings, os
from pathlib import Path

from typing import Dict
import duckdb, dlt, pandas as pd
import plotly.graph_objects as go

from stables.data.source.defillama import defillama_yield_pool

warnings.filterwarnings('ignore')

if Path.cwd().name == "notebooks":
    rootdir = Path.cwd().parent
else:
    rootdir = Path.cwd()

duckdb_destination = str(rootdir / "data/raw/raw_defillama.duckdb")


def load_yield_pool(pool_id: str, table_name: str):
    """
    Get all yield pools from DefiLlama
    """
    pipeline = dlt.pipeline(
        pipeline_name="defillama",
        destination=dlt.destinations.duckdb(duckdb_destination),
        dataset_name="yields",
    )

    load_info = pipeline.run(defillama_yield_pool(pool_id), table_name=table_name, write_disposition="replace")
    return load_info

def get_yield_pool_df(table_name: str, duckdb_destination: str=duckdb_destination, table_schema: str="yields"):
    """
    Load the yield pool data from duckdb
    """
    sql = f"SELECT * FROM {table_schema}.{table_name}"
    conn = duckdb.connect(duckdb_destination)
    df = conn.execute(sql).fetchdf()
    return df

class StableVisualizer:
    """Create visualizations for stablecoin data."""

    @staticmethod
    def plot_yields(
        pools_data: Dict[str, pd.DataFrame],
        title: str = "Yields",
        days: int = 90,
    ) -> go.Figure:
        """
        Create a plot showing APY and TVL trends for multiple pools.

        Args:
            pools_data: Dictionary mapping pool names to their yield data DataFrames
            title: Title for the plot

        Returns:
            Plotly figure object with two subplots (APY and TVL)
        """
        # Create figure with secondary y-axis
        fig = go.Figure()

        # Calculate the date N days ago in UTC
        days_ago = pd.Timestamp.now(tz="UTC") - pd.Timedelta(days=days)

        # Plot each pool's data
        for pool_name, df in pools_data.items():
            # Convert timestamp to datetime if it's not already
            df["timestamp"] = pd.to_datetime(df["timestamp"])

            # Filter data for last N days
            df = df[df["timestamp"] >= days_ago]

            # Add APY trace
            fig.add_trace(
                go.Scatter(
                    x=df["timestamp"],
                    y=df["apy"],
                    name=f"{pool_name}",
                    line=dict(width=2),
                )
            )

        # Update layout with two y-axes
        fig.update_layout(
            title=title,
            yaxis=dict(
                title="APY (%)", side="left", showgrid=True, gridcolor="lightgrey"
            ),
            hovermode="x unified",
            showlegend=True,
            legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.05),
            template="plotly_white",
            xaxis=dict(range=[days_ago, pd.Timestamp.now(tz="UTC")]),
        )

        return fig

In [ ]:
with open(rootdir / "data/address/defillama_pools.json", "r") as f:
    ybs_pools = json.load(f)

for pool_name, pool_id in ybs_pools.items():
    load_yield_pool(pool_id, pool_name.lower())

ybs_yields = {
    symbol: get_yield_pool_df(table_name=symbol.lower())
    for symbol in ybs_pools
}

fig = StableVisualizer.plot_yields(ybs_yields)
fig.show()